In [1]:
import pandas as pd

In [2]:
# Wczytanie danych od Shumee
df_org = pd.read_excel('shumee_mckinsey -Aktualizacja 01.03.xlsx', index_col=None, engine='openpyxl') 

In [3]:
def finder(phrase):
    return (lambda product_name : str(product_name).find(phrase))

df = df_org[df_org['ID zamówienia'].notna()] # usunięcie wpisów gdzie ID zamówienia == NaN
df = df[df['Nazwa produktu'] != 'Przesyłka pobraniowa']
df = df[df['Nazwa produktu'].map(finder("Kod rabatowy")) == -1] # usuniecie kodow rabatowych
df = df[df['Miasto'] != 'test']
df = df[df['Miasto'] != 'Test']
df = df[df['Miasto'] != 'TEST']
df = df[df['Miasto'] != 'ssss']
df = df[df['Miasto'] != 'asd']
df = df[df['Miasto'] != 'asdasd']
df = df[df['Miasto'] != 'dsss']
df = df[df['Nazwa produktu'].map(finder("TESTOWY PRODUKT")) == -1]
df = df[df['Nazwa produktu'].map(finder("PRODUKT TESTOWY")) == -1]
df = df[df['Cena'] >= 0] # usuniecie kodow rabatowych
# akceptuję produkty z ceną 0 zł, ponieważ są to produkty zamówione osobiście lub też telefonicznie

Na razie ograniczę się tylko do produktów sprzedanych w Polsce:

In [4]:
df = df[df["Kraj"]=="PL"]

Sprawdzam poprawność danych:

In [5]:
print("Liczba NaN w każdej kolumnie: ")
df.isna().agg("sum")

Liczba NaN w każdej kolumnie: 


ID zamówienia         0
Data                  0
Źródło              229
Kraj                  0
Miasto              270
Kod Pocztowy        291
Nazwa produktu        0
SKU               12613
EAN               13725
Ilość                 0
Cena                  0
Waluta                0
Koszt dostawy         0
Forma dostawy      3432
dtype: int64

Teraz załaduję model Word2Vec. Wykorzsytuję bibliotekę Gensim, oraz model wytrenowany m.in. na polskiej Wikipedii, który można pobrać pod tym adresem: https://github.com/sdadas/polish-nlp-resources

In [6]:
# !pip install --upgrade gensim

from gensim.models import KeyedVectors

word2vec = KeyedVectors.load("./word2vec/word2vec_100_3_polish.bin")

C:\Users\PiotrGrabysz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Word2Vec zamienia słowa na wektory (w tym przypadku wektory wymiaru 100). Na przykład słowu "krzesło" odpowiada taki wektor:

In [7]:
word2vec["krzesło"]

array([ 2.012289e+00,  1.462072e+00, -5.004100e+00, -3.441711e+00,
        3.990560e-01, -2.565111e+00,  5.030282e+00,  6.047542e+00,
        4.548630e+00,  1.738084e+00, -2.758710e-01, -7.522800e-02,
        1.855085e+00, -6.247217e+00, -5.888261e+00, -8.400401e+00,
       -2.965125e+00, -3.482589e+00,  4.830710e-01, -1.018867e+00,
       -8.132490e-01, -3.048696e+00,  4.958340e+00, -1.293860e-01,
       -3.085849e+00, -1.863563e+00, -5.953629e+00,  4.385289e+00,
        1.279462e+00, -1.170168e+00, -8.543170e-01, -4.681480e-01,
        5.515535e+00, -7.276000e-03,  7.161128e+00,  3.059943e+00,
        2.034154e+00,  3.861800e-01,  2.643000e+00, -7.206631e+00,
       -2.734934e+00,  4.014585e+00,  3.858501e+00, -5.677879e+00,
        4.270895e+00,  6.786670e-01,  3.034365e+00,  1.110994e+00,
       -4.248924e+00,  2.391838e+00,  2.946400e-02,  1.036666e+00,
        5.377800e-01, -5.942716e+00, -4.273711e+00, -1.379860e+00,
        2.706288e+00,  2.586019e+00,  3.046250e+00,  1.442850e

Na podstawie takiej wektorowej reprezentacji można wyszukać słowa o najbardziej podobnym znaczeniu do danego słowa.

Słowa podobne do "krzesło":

In [8]:
print(word2vec.similar_by_word("krzesło"))

[('stołek', 0.9247931241989136), ('taboret', 0.9213142991065979), ('fotel', 0.9031204581260681), ('kanapa', 0.8804746270179749), ('krzesełko', 0.8769845962524414), ('sofa', 0.8673840165138245), ('zydel', 0.8485152125358582), ('tapczan', 0.8382896780967712), ('otomana', 0.8218340277671814), ('szezlong', 0.8153223991394043)]


C:\Users\PiotrGrabysz\anaconda3\lib\site-packages\gensim\models\keyedvectors.py:772: RuntimeWarning: invalid value encountered in true_divide
  dists = dot(self.vectors[clip_start:clip_end], mean) / self.norms[clip_start:clip_end]


Oraz słowa podobne do słowa "komputer"

In [9]:
print(word2vec.similar_by_word("komputer"))

[('oprogramowanie', 0.799543559551239), ('sterownik', 0.794553279876709), ('kalkulator', 0.7934095859527588), ('skaner', 0.7867334485054016), ('urządzenie', 0.7822620868682861), ('laptop', 0.7730702757835388), ('drukarka', 0.7705292105674744), ('procesor', 0.7685826420783997), ('czytnik', 0.7645211219787598), ('serwer', 0.7605681419372559)]


Tworzę zbiór wszystkich słów wykorzystanych w nazwach produktów. Wszystkie wielkie litery zastępuję małymi literami, bo tylko takie słowa są rozpoznawane przez word2vec.

In [8]:
words_set = set()

for name in df["Nazwa produktu"].to_list():
    for word in name.lower().split():
        words_set.add(word)

In [9]:
len(words_set)

33669

W nazwach produktów użyto 33669 różnych słów. Być może niektóre z nich, np oznaczające wymiary (jak 10x10), trzeba będzie odrzucić.

In [10]:
# Słowa które znalazły się w banku słów word2vec:
included_words = []
# Słowa których word2vec nie zna
removed_words = []
# lista wektorów odpowiadających poszeczególnym słowom
words_array = []

for word in words_set:
    try:
        vec = word2vec[word]
        words_array.append(vec)
        included_words.append(word)
    except KeyError:
        removed_words.append(word)

In [11]:
print(f"Udało się zakodować {len(included_words)}")
print(f"{len(removed_words)} słów nie znalazło się w bazie słów")

Udało się zakodować 9175
24494 słów nie znalazło się w bazie słów


In [12]:
import numpy as np

words_array = np.array(words_array)

### Klastrowanie za pomocą DBSCAN

In [13]:
from sklearn.cluster import DBSCAN
import collections

In [53]:
clustering = DBSCAN(eps=0.22, min_samples = 5, metric = "cosine", algorithm="brute").fit(words_array)

In [56]:
print("Klastry: ")
print(set((clustering.labels_)))

Klastry: 
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, -1}


In [58]:
# Słownik w którym kluczami są słowa, a wartościami numer klastra w którym znajduje się to słowo
word2cluster = dict()

# Słownik w którym kluczem jest numer klastra, a wartością lista słów w klastrze
cluster2word = collections.defaultdict(list)

for idx, word in enumerate(included_words):
    word2cluster[word] = clustering.labels_[idx]
    cluster2word[clustering.labels_[idx]].append(word)

In [59]:
print(f"Number of outliers: {len(cluster2word[-1])}")
for i in range(len(cluster2word) - 1):
    print(f"cluster {i}: {len(cluster2word[i])}")

Number of outliers: 6718
cluster 0: 762
cluster 1: 22
cluster 2: 181
cluster 3: 677
cluster 4: 269
cluster 5: 12
cluster 6: 61
cluster 7: 6
cluster 8: 6
cluster 9: 10
cluster 10: 82
cluster 11: 9
cluster 12: 14
cluster 13: 11
cluster 14: 10
cluster 15: 62
cluster 16: 20
cluster 17: 16
cluster 18: 5
cluster 19: 7
cluster 20: 6
cluster 21: 5
cluster 22: 26
cluster 23: 6
cluster 24: 5
cluster 25: 6
cluster 26: 8
cluster 27: 8
cluster 28: 6
cluster 29: 7
cluster 30: 12
cluster 31: 18
cluster 32: 7
cluster 33: 13
cluster 34: 5
cluster 35: 6
cluster 36: 5
cluster 37: 8
cluster 38: 10
cluster 39: 4
cluster 40: 4
cluster 41: 4
cluster 42: 3
cluster 43: 4
cluster 44: 3
cluster 45: 6
cluster 46: 6
cluster 47: 5
cluster 48: 5
cluster 49: 4


Problem jest taki, że niektóre klastry są bardzo małe. Z drugiej strony niektóre z nich wydaja się bardzo sensowne:

In [18]:
def print_cluster(cluster):
    words_long_string = ""
    for word in cluster:
        words_long_string += word
        words_long_string += ", "
    print(words_long_string)

In [44]:
for i in range(len(cluster2word) - 1):
    print(f"Cluster {i}. Core sample {}")
    print_cluster(cluster2word[i])
    print("\n")

cluster 0: 
manometr, plecionka, oberżyna, ruszt, woreczek, linksys, pasek, garnek, leżak, przewodowy, stół, miód, peleryna, zielono, podnośnik, pisuar, notes, czerwono, turkusowo, napój, koronka, palczatka, welurowy, aksamitny, pręt, kurtka, szorty, ravioli, dżins, jasnoczerwony, szuflada, kamizelka, pudło, naczynia, haczyk, wiaderko, czarno, błękitny, sztruksowy, sosna, sekretarzyk, spryskiwacz, olejowy, półeczka, koniczyna, ceownik, wełniany, majtki, pączek, spodenki, turkusowy, szafka, dąb, łyżeczka, szaty, klawisze, figowiec, ananas, jasnobrązowy, ciemnofioletowy, kuchenka, deserowy, joystick, mango, pisak, samsung, mufa, ciemnoniebieski, chlebak, bluetooth, kamelia, nadajnik, dzbanek, falbanka, świder, pieczenia, transmiter, jednofazowy, sygnalizator, krzesło, klinga, kostium, kasztan, talerzyk, talerz, nauszniki, koronkowy, polarowy, czapka, satyna, przewód, nożyce, podkoszulek, haft, bolerko, ręcznik, kalafior, dywan, rododendron, otomana, słoik, kwiat, piecyk, wkrętarka, dres,

Jakie słowa znalazły się wśród outlierów?

In [45]:
print_cluster(cluster2word[-1])

stm, nastawny, mys, teraz, wysoki, revlon, korkowy, sonoma, madon, koci, krople, podtynkowy, niełamliwy, przepływ, dym, simba, badge, ced, aurita, inspi, balm, muszka, elf, dwuobwodowy, escape, lurex, zapf, saffi, 0925, donald, afera, trener, visone, dźwięk, śnieżka, militarny, nadruk, zoch, sky, przeciwłupieżowy, mot, ar, clix, jednokomorowy, przyssa, oszczędzacz, pma, siłacz, bejsbolówka, burito, gruba, nitro, neo, mdm, holmes, 3730, paste, amita, stalow, tons, abażur, gimnastyczny, korytko, grafit, velda, power, wibrator, winter, wenecja, spc, ślub, capital, oryginalny, meccano, vt, materiałowy, citizen, bat, poker, drzw, double, brąz, alicante, dover, grzybkowy, shiatsu, book, awex, cfl, królewski, branch, airmax, laski, uht, soka, permanent, miszuk, imitacja, boxe, merveilles, parowar, śliweczka, ro, kredki, pmma, oświetleniowy, torre, robiony, bar, bond, callatis, bukow, drewniany, wishbone, zakrywka, moxy, podra, duży, stojąca, uralny, 3236, kapitana, sofia, 6789, oud, rapid, zj